In [1]:
from pathlib import Path

import numpy as np
from skimage import io
from skimage.transform import rescale, resize
from scipy.ndimage import gaussian_filter
from scipy import ndimage

import torch
# from transformers import Dinov2Backbone
from torchvision.models import resnet18

/home/iternal/miniconda3/envs/rerun/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/iternal/miniconda3/envs/rerun/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
code_dir = Path.home() / "work/brox/python-example-droid-dataset"

In [3]:
backbone = resnet18(pretrained=True)

/home/iternal/miniconda3/envs/rerun/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/iternal/miniconda3/envs/rerun/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
children = list(backbone.children())
newmodel = torch.nn.Sequential(*children[0:6])
# newmodel

In [14]:
imginfo = lambda img: print(type(img), img.dtype, img.shape, img.min(), img.max())

# === load first touch coors
import json
with open(code_dir / "data/single_log.json") as f:
    episode_log = json.load(f)
first_x, first_y, first_z = map(int, episode_log["first_touch"])
cropw = 200

In [21]:
# === first image VS max distant image during grip
data_path = code_dir / "data/frames"
file_list = sorted(data_path.glob("center*jpg"))
first_image = io.imread(file_list[0])
mid_image = io.imread(file_list[-1])
max_image = io.imread(data_path / "max_image.jpg")

h, w = first_image.shape[:2]
if (0 <= first_y - cropw and first_y + cropw < h and
    0 <= first_x - cropw and first_x + cropw < w):
    good_episode = True

In [22]:
def to_batch(image):
    downscaled = resize(image, (224, 224))
    batch = torch.from_numpy(downscaled)[None, :, :, :].to(torch.float32) / 255  # [h, w, c] -> [1, h, w, c]
    batch = batch.permute(0, 3, 1, 2)  # [1, h, w, c] -> [1, c, h, w]
    return batch

def cut_crop(image):
    """
    :param image: [h, w, c]
    """
    return image[first_y-cropw:first_y+cropw, first_x-cropw:first_x+cropw]

save_images = {
    "crop_first": cut_crop(first_image),
    "crop_max": cut_crop(max_image)
}

for name, image in save_images.items():
    io.imsave(data_path / (name + ".jpg"), image.astype(np.uint8))

In [23]:
# === Run resnet on crops
dino1 = newmodel(to_batch(cut_crop(first_image))).detach() # [1, c, h, w] range [0, 1] torch.float32
dino2 = newmodel(to_batch(cut_crop(max_image))).detach()

def scale(diff_image: np.ndarray):
    low, high = np.quantile(diff_image, 0.1), np.quantile(diff_image, 0.99)
    return np.clip(diff_image / high * 255, 0, 255)

diff = dino1 - dino2
diff = (diff[0, :, :, :] ** 2).sum(axis=0).numpy() # [h, w]

dino_mid = newmodel(to_batch(cut_crop(mid_image))).detach()
diff_mid = ((dino_mid - dino2)[0, :, :, :] ** 2).sum(axis=0).numpy()
diff_mid = gaussian_filter(diff_mid, sigma=1) * 2
diff = diff - diff_mid

diff = resize(diff, (cropw*2, cropw*2))

io.imsave(data_path / "result_diff.jpg", scale(diff).astype(np.uint8))

In [24]:
def overlay_crop(canvas: np.ndarray, diff: np.ndarray):
    diff = scale(resize(diff, (cropw*2, cropw*2))).astype(np.uint8)
    diff = np.tile(diff[:, :, None], (1, 1, 3))  # 1 channel to 3 channels
    diff[:, :, 1] = 0
    diff[:, :, 2] = 0
    
    c = np.copy(canvas)
    canvas_crop = c[first_y-cropw:first_y+cropw, first_x-cropw:first_x+cropw]
    c[first_y-cropw:first_y+cropw, first_x-cropw:first_x+cropw] = canvas_crop * 0.4 + diff * 0.6 # use broadcasting
    return c

io.imsave(data_path / "result_overlay.jpg", overlay_crop(first_image, diff))

## Simple difference

In [41]:
# === Simple difference
imginfo(first_image)
diff = crop(first_image.astype(np.float32)) - crop(max_image.astype(np.float32)) # careful, must be float!
diff = np.sum(diff ** 2, axis=2)
io.imsave("data/result_simple.jpg", resize(scale(diff), crop_shape[:2]).astype(np.uint8))

<class 'numpy.ndarray'> uint8 (720, 1280, 3) 0 255


NameError: name 'crop' is not defined

In [15]:
data_path = Path("data")
file_list = sorted(data_path.glob("before_*jpg"))
before_image = io.imread(file_list[-3])

file_list = sorted(data_path.glob("after_*jpg"))
after_image = io.imread(file_list[3])

grip_image = io.imread("data/grip_image.jpg")

In [29]:
# === Simple difference
diff = crop(grip_image.astype(np.float32)) - crop(before_image.astype(np.float32)) # careful, must be float!
diff = np.sum(diff ** 2, axis=2)
diff[diff < 10000] = 0
diff[diff > 10000] = 1

diff_2 = crop(grip_image.astype(np.float32)) - crop(after_image.astype(np.float32)) # careful, must be float!
diff_2 = np.sum(diff_2 ** 2, axis=2)

#diff_2 = diff_2 - diff * 100
io.imsave("data/result_0.jpg", resize(scale(diff), crop_shape[:2]).astype(np.uint8))
io.imsave("data/result_1.jpg", resize(scale(diff_2), crop_shape[:2]).astype(np.uint8))


In [18]:
io.imsave("data/a1.jpg", crop(before_image).astype(np.uint8))
io.imsave("data/a2.jpg", crop(grip_image).astype(np.uint8))
io.imsave("data/a3.jpg", crop(after_image).astype(np.uint8))